# Megaline, prediccion de planes

La compañía móvil Megaline se enfrenta a un desafío importante: un número significativo de sus clientes aún utiliza planes heredados, lo que puede limitar tanto la satisfacción del cliente como la eficiencia operativa de la empresa. Con el objetivo de optimizar la experiencia del cliente y alentar la transición a sus nuevos planes, Smart y Ultra, Megaline busca desarrollar un modelo predictivo capaz de analizar el comportamiento de los clientes y recomendar el plan más adecuado.

En este proyecto, se cuenta con acceso a los datos de comportamiento de suscriptores que ya han migrado a los nuevos planes, obtenidos del sprint de Análisis estadístico de datos. Utilizando esta información, se debe crear un modelo de clasificación que determine con precisión cuál de los nuevos planes es el más apropiado para cada cliente.

La tarea principal es desarrollar un modelo que alcance la mayor exactitud posible, con un umbral mínimo de exactitud establecido en 0.75. Se utilizará el dataset disponible para validar y comprobar la exactitud del modelo. 

## Importacion de datos y librerias

In [22]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
import numpy as np


In [23]:
df = pd.read_csv('users_behavior.csv')

In [24]:
display(df.head())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [25]:
df['is_ultra'].value_counts(1)

is_ultra
0    0.693528
1    0.306472
Name: proportion, dtype: float64

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Los datos se observan correctos y sus tipos tambien, no hay valores ausentes.

## Division del Dataset

In [27]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

In [28]:
# division del dataset 60% 20% 20% 
features_train, feature_temp, target_train, target_temp = train_test_split(features, target, test_size=0.40, random_state=71, stratify=target)

In [29]:
features_test, features_valid, target_test, target_valid = train_test_split(feature_temp, target_temp, test_size=0.50, random_state=71, stratify=target_temp)

In [30]:
target_train.value_counts(1)

is_ultra
0    0.693465
1    0.306535
Name: proportion, dtype: float64

In [31]:
target_test.value_counts(1)

is_ultra
0    0.693624
1    0.306376
Name: proportion, dtype: float64

In [32]:
target_valid.value_counts(1)

is_ultra
0    0.693624
1    0.306376
Name: proportion, dtype: float64

In [33]:
features_train.info()
features_test.info()
features_valid.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1928 entries, 894 to 1679
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     1928 non-null   float64
 1   minutes   1928 non-null   float64
 2   messages  1928 non-null   float64
 3   mb_used   1928 non-null   float64
dtypes: float64(4)
memory usage: 75.3 KB
<class 'pandas.core.frame.DataFrame'>
Index: 643 entries, 1944 to 3053
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     643 non-null    float64
 1   minutes   643 non-null    float64
 2   messages  643 non-null    float64
 3   mb_used   643 non-null    float64
dtypes: float64(4)
memory usage: 25.1 KB
<class 'pandas.core.frame.DataFrame'>
Index: 643 entries, 725 to 3066
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     643 non-null    float64
 1   minutes   643 non-nu

Dado que desconocemos si en un futuro cercano existira un dataset en el cual se pueda testear el modelo. Se divide el dataset con el que se cuenta en un porcentaje de 60% para el dataset de entrenamiento, 20% para el set de validacion y 20% para el set de testeo.

## Arbol de decision

In [34]:
best_score_tree = 0
best_depth = 0

for depth in range(1,11):
    model = DecisionTreeClassifier(max_depth=depth, random_state=71, class_weight='balanced')
    model.fit(features_train, target_train)
    
    train_score = model.score(features_train, target_train)
    #train_predictions = model.predict(features_train)
    #train_accuracy = accuracy_score(target_train, train_predictions)

    valid_score = model.score(features_valid, target_valid)   
    #test_predictions = model.predict(features_test)
    #test_accuracy = accuracy_score(target_test, test_predictions)

    print("Exactitud de max_depth igual a: ", depth, ": ")
    print("Conjunto de entrenamiento: ", train_score)
    print("Conjunto de validacion: ", valid_score, "\n")

    if valid_score > best_score_tree:

        best_score_tree = valid_score
        best_depth = depth

print('La exactitud del mejor modelo en el conjunto de validacion (depth={}) :{}'.format(best_depth, best_score_tree))

# 3
    #print("Conjunto de entrenamiento: ", train_accuracy)
    #print("Conjunto de prueba: ", test_accuracy)


Exactitud de max_depth igual a:  1 : 
Conjunto de entrenamiento:  0.7505186721991701
Conjunto de validacion:  0.7620528771384136 

Exactitud de max_depth igual a:  2 : 
Conjunto de entrenamiento:  0.7733402489626556
Conjunto de validacion:  0.7729393468118196 

Exactitud de max_depth igual a:  3 : 
Conjunto de entrenamiento:  0.7852697095435685
Conjunto de validacion:  0.7838258164852255 

Exactitud de max_depth igual a:  4 : 
Conjunto de entrenamiento:  0.7427385892116183
Conjunto de validacion:  0.744945567651633 

Exactitud de max_depth igual a:  5 : 
Conjunto de entrenamiento:  0.8075726141078838
Conjunto de validacion:  0.7869362363919129 

Exactitud de max_depth igual a:  6 : 
Conjunto de entrenamiento:  0.8184647302904564
Conjunto de validacion:  0.7791601866251944 

Exactitud de max_depth igual a:  7 : 
Conjunto de entrenamiento:  0.8319502074688797
Conjunto de validacion:  0.8009331259720062 

Exactitud de max_depth igual a:  8 : 
Conjunto de entrenamiento:  0.8163900414937759

### Observaciones

La mejor exactitud en el conjunto de validación es 0.8009 con una profundidad máxima de 7, lo cual supera el umbral de 0.75 marcado como minimo necesario para el proyecto. La exactitud en el conjunto de entrenamiento con max_depth=7 es 0.8319, indicando un alto rendimiento en los datos de entrenamiento.

En los valores mencionados se observa un sobreajuste de 0.3 punto, por lo que no seria la mejor eleccion. Al observar con mayor detenimiento los resultados, es posible verificar que la profundidad donde menos sobreajuste se presenta es en 3, donde la diferencia es de 0.0015 y los resultados en el test de validacion siguen siendo superiores al minimo requerido, con un 0.7838.

## Bosque aleatorio

In [35]:
best_score_forest = 0
best_est = 0

for est in range(1,11):
    model_forest = RandomForestClassifier(n_estimators=est, random_state=71)
    model_forest.fit(features_train, target_train)

    train_score_forest = model_forest.score(features_train, target_train)

    valid_score_forest = model_forest.score(features_valid, target_valid)   
    

    print("Exactitud de numero de estimadores igual a: ", est, ": ")
    print("Conjunto de entrenamiento: ", train_score_forest)
    print("Conjunto de prueba: ", valid_score_forest, "\n")

    if valid_score_forest > best_score_forest:
        best_score_forest = valid_score_forest
        best_est = est

print('La exactitud del mejor modelo en el conjunto de validacion (n_estimators={}) :{}'.format(best_est, best_score_forest))

#6


Exactitud de numero de estimadores igual a:  1 : 
Conjunto de entrenamiento:  0.8890041493775933
Conjunto de prueba:  0.7340590979782271 

Exactitud de numero de estimadores igual a:  2 : 
Conjunto de entrenamiento:  0.904045643153527
Conjunto de prueba:  0.7729393468118196 

Exactitud de numero de estimadores igual a:  3 : 
Conjunto de entrenamiento:  0.941908713692946
Conjunto de prueba:  0.7791601866251944 

Exactitud de numero de estimadores igual a:  4 : 
Conjunto de entrenamiento:  0.9408713692946058
Conjunto de prueba:  0.7947122861586314 

Exactitud de numero de estimadores igual a:  5 : 
Conjunto de entrenamiento:  0.9585062240663901
Conjunto de prueba:  0.7962674961119751 

Exactitud de numero de estimadores igual a:  6 : 
Conjunto de entrenamiento:  0.9564315352697096
Conjunto de prueba:  0.8055987558320373 

Exactitud de numero de estimadores igual a:  7 : 
Conjunto de entrenamiento:  0.9766597510373444
Conjunto de prueba:  0.8009331259720062 

Exactitud de numero de estima

### Observaciones

La mejor exactitud en el conjunto de validación es 0.8071 con 8 estimadores. La exactitud en el conjunto de entrenamiento con 8 estimadores es 0.9688, mostrando nuevamente un sobreajuste importante de 0.16 puntos. Por lo que esta tampoco parece ser la opcion mas sensata

La menor diferencia es de 0.1311 con 2 estimadores, pero la precisión en validación es menor 0.7729 en comparación con el árbol de decisión max_depth = 3.

## Regresion logistica

In [36]:

model_linear = LogisticRegression(random_state=71, solver='liblinear')
model_linear.fit(features_train, target_train)

train_score_linear = model_linear.score(features_train, target_train)

valid_score_linear = model_linear.score(features_valid, target_valid)   
    
print("La exactitud del modelo de regresión logística en el conjunto de entrenamiento:", train_score_linear)
print("La exactitud del modelo de regresión logística en el conjunto de validación:", valid_score_forest)

La exactitud del modelo de regresión logística en el conjunto de entrenamiento: 0.7033195020746889
La exactitud del modelo de regresión logística en el conjunto de validación: 0.8055987558320373


### Observaciones 

Muestra subajuste con una precisión en entrenamiento relativamente baja (0.7033), pero tiene una buena precisión en validación (0.8056). Sin embargo, la gran diferencia negativa sugiere que no está capturando bien los patrones en los datos.

## Conclusion 

Modelo Elegido: Arbol de desicion con una fofundidad de 3.

Razón:  Este modelo tiene una buena precisión en validación (0.7838) y una diferencia mínima entre los conjuntos de entrenamiento y validación, lo que indica un buen equilibrio entre complejidad del modelo y capacidad de generalización.

## Calidad del modelo 

In [37]:
model_test = DecisionTreeClassifier(random_state=71, max_depth= 3)

model_test.fit(features_train, target_train)

predictions = model_test.predict(features_test)

errors = (predictions != target_test.values).sum()

score_t = model_test.score(features_test, target_test)

print('La exactitud del modelo es de: ',score_t)

print('El numero de errores cometidos por el modelo es de: ', errors)

print('El porcentaje de errores cometidos por el modelo es de: ', (errors/len(target_test))*100, '%')

La exactitud del modelo es de:  0.7822706065318819
El numero de errores cometidos por el modelo es de:  140
El porcentaje de errores cometidos por el modelo es de:  21.77293934681182 %


## Prueba de cordura

In [38]:

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_train, target_train)

dummy_predictions = dummy_clf.predict(features_test)

score_c = dummy_clf.score(features_test, target_test)

print(score_c)


0.6936236391912908


## Conclusiones Generales

1. Se realizaron pruebas con tres distintos tipo de modelos para clasificacion, arbol de decision, bosque aleatorio y regresion logitica. En los tres casos se observa que los modelos se ven afectados por el sobreajuste, por esta razon se elige como modelo para realizar el teste el Arbol de decision con una profundidad de 3; dado que es el que presenta la menor diferencia en la exactitud caculada entre el set de entrenamiento y el de validacion. 

2. Se entreno nuevamente el modelo con los parametros elegidos y para que generara las predicciones sobre el set de testeo. Al calcular su exactitud se observa que obtuvo una puntuacion del 0.7822, por lo que super el 0.75 establecido como minimo requerido. 

3. Se realizo una prueba de cordura con el modelo DummyClassifier y se obtuvo una exactitud del 0.6936, por lo que se puede concluir que el modelo elegido es capaz de predecir con mayor exactitud que la casualidad o si lo dejaramos a la suerte.

4. Se realizo el calculo de los errores que cometio el modelo en el set de testeo y se observa que 140 resultados de los 643 son errores. Lo que corresponde al 21% y concuerda con el porcentaje de aciertos. 

5. Dados los resultados obtenidos podemos considerar que el modelo funciona en una parte importante de los pasos y ayudaria a predecir con una exactitud del 78% 